In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import socket as socket
import os as os
import sys as sys
import h5py
import matplotlib.cm as cm
import itertools as it
import multiprocessing as mp

socket_name = socket.gethostname()
print(socket_name)

    
if socket_name.startswith("bionc"):
    print("Leipzig Cluster detected!")
    path = "/mnt/archgen/users/hringbauer/brienzi/"
    #sys.path.append("/mnt/archgen/users/hringbauer/data/malta/") 
    
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")
    
os.chdir(path)  # Set the right Path (in line with Atom default)
print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")

bionc21
Leipzig Cluster detected!
/mnt/archgen/users/hringbauer/brienzi
CPU Count: 40


In [4]:
p = "/mnt/archgen/users/hringbauer/brienzi/data/splitbam/Brienzi.trio.lib.vcf.gz"

In [6]:
!bcftools query -l $p 

/mnt/archgen/users/hringbauer/brienzi/data/splitbam/A55903.bam
/mnt/archgen/users/hringbauer/brienzi/data/splitbam/A55904.bam
/mnt/archgen/users/hringbauer/brienzi/data/splitbam/Brienzi.merged.bam


In [8]:
!bcftools query -f '%POS\n' $p | wc -l

78397799


## Test downsampled 1240k set VCF

In [10]:
v = "0.1"
ch=22

p = f"/mnt/archgen/users/hringbauer/brienzi/data/vcf1240k.imputed/v{v}.imputed.ch{ch}.vcf.gz"
!bcftools query -f '%POS\n' $p | wc -l

15976


In [11]:
!bcftools query -l $p 

/mnt/archgen/users/hringbauer/brienzi/data/splitbam/A55903.bam
/mnt/archgen/users/hringbauer/brienzi/data/splitbam/A55904.bam
/mnt/archgen/users/hringbauer/brienzi/data/splitbam/Brienzi.merged.bam


In [ ]:
!bcftools view $p | head -1000

### Explore H5 File

In [18]:
h5_path = "/mnt/archgen/users/hringbauer/brienzi/data/h5.imputed/v0.1.imputed.ch"
ch =3

with h5py.File(f"{h5_path}{ch}.h5", "r") as f: # Load for Sanity Check. See below!
        print(list(f["variants"]))
        print(np.shape(f["calldata/GT"]))
        #gt = f["calldata/GT"][:]
        gp = f["calldata/GP"][:][:,:,:] # Fill in SNP number for TS
        samples = f["samples"][:].astype("str")

['AF', 'AF_ALL', 'AF_SAMPLE', 'ALT', 'BUF', 'CHROM', 'FILTER_PASS', 'ID', 'INFO', 'MAP', 'POS', 'QUAL', 'RAF', 'REF', 'altlen', 'is_snp', 'numalt']
(78525, 3, 2)


In [21]:
gp_max = np.max(gp, axis=2)

In [25]:
np.mean(gp_max>=0.99, axis=0)

array([0.03766953, 0.03766953, 0.03766953])

In [26]:
gp_max

array([[0.488, 0.488, 0.488],
       [0.613, 0.613, 0.613],
       [0.694, 0.694, 0.694],
       ...,
       [0.978, 0.978, 0.978],
       [0.482, 0.482, 0.482],
       [0.654, 0.654, 0.654]], dtype=float32)

In [63]:
h5_path = "/mnt/archgen/users/hringbauer/brienzi/data/h5/BRI001.chr"
ch =3

with h5py.File(f"{h5_path}{ch}.hdf5", "r") as f: # Load for Sanity Check. See below!
        print(list(f["variants"]))
        print(list(f["calldata"]))
        print(np.shape(f["calldata/AD"]))
        ad = f["calldata/AD"][:]
        gt = f["calldata/GT"][:]
        #gp = f["calldata/GP"][:][:,:,:] # Fill in SNP number for TS
        samples = f["samples"][:].astype("str")

['ALT', 'MAP', 'POS', 'REF']
['AD', 'GT']
(77652, 1, 2)


In [64]:
np.mean(np.sum(ad, axis=2))

4.725609127903982

In [61]:
np.mean(np.sum(ad, axis=2))

4.762916602276825

In [65]:
np.mean(np.sum(ad, axis=2)>0)

0.671573172616288

In [62]:
np.mean(np.sum(ad, axis=2)>0)

0.5745119249987122

### How many reads in file

In [85]:
p = "/mnt/archgen/users/hringbauer/brienzi/data/splitbam/A55903.bam" # Y capture

In [82]:
!samtools view -c $p

28740097


In [34]:
!samtools view -c $p

29474192


In [84]:
!samtools idxstats $p | cut -f 1,3

1	1259767
2	1260330
3	1186454
4	1249100
5	1137855
6	1012158
7	841791
8	891338
9	643083
10	694954
11	811359
12	761135
13	512535
14	488362
15	395600
16	328505
17	286900
18	359662
19	260578
20	241819
21	174950
22	129557
X	1051198
Y	2640755
MT	45852
GL000207.1	7
GL000226.1	935
GL000229.1	111
GL000231.1	928
GL000210.1	31
GL000239.1	191
GL000235.1	932
GL000201.1	43
GL000247.1	79
GL000245.1	1167
GL000197.1	55
GL000203.1	30
GL000246.1	605
GL000249.1	35
GL000196.1	573
GL000248.1	53
GL000244.1	53
GL000238.1	77
GL000202.1	62
GL000234.1	908
GL000232.1	332
GL000206.1	48
GL000240.1	959
GL000236.1	658
GL000241.1	566
GL000243.1	121
GL000242.1	39
GL000230.1	161
GL000237.1	289
GL000233.1	589
GL000204.1	123
GL000198.1	720
GL000208.1	3393
GL000191.1	236
GL000227.1	477
GL000228.1	222
GL000214.1	4760
GL000221.1	3603
GL000209.1	94
GL000218.1	2982
GL000220.1	9881
GL000213.1	471
GL000211.1	1974
GL000199.1	759
GL000217.1	1121
GL000216.1	501
GL000215.1	622
GL000205.1	2568
GL000219.1	4410
GL000224.1	1783
GL000223

In [74]:
!samtools view $p | head -n 1000000 | cut -f 10 | perl -ne 'chomp;print length($_) . "\n"' | sort | uniq -c

samtools view: writing to standard output failed: Broken pipe
samtools view: error closing standard output: -1
   4331 10
   6706 11
   8535 12
   9133 13
   9608 14
   9731 15
   9068 16
   9009 17
   8915 18
   8317 19
   7866 20
   6732 21
   6124 22
   4646 23
   3572 24
   2611 25
   1753 26
   1308 27
   1040 28
    883 29
   1059 30
   1026 31
   1108 32
   1230 33
   1265 34
   1442 35
   1658 36
   1662 37
   2009 38
   2148 39
   2554 40
   2718 41
   2825 42
   2980 43
   2990 44
   3254 45
   3499 46
   3661 47
   3858 48
   4199 49
      2 5
   4566 50
   4793 51
   5015 52
   5105 53
   4973 54
   5047 55
   5180 56
   5393 57
   5562 58
   5867 59
      6 6
   5880 60
   6269 61
   6162 62
   6437 63
   6302 64
   6220 65
   6238 66
   6078 67
   6541 68
   6623 69
    137 7
   6843 70
   7666 71
   9627 72
  22186 73
 668851 76
    950 8
   2448 9


In [78]:
p = '/mnt/archgen/users/hringbauer/data/brienzi_capture/A55903.bam'
!samtools view -c $p

19183425


In [79]:
p = "/mnt/archgen/users/hringbauer/brienzi/data/capture/A55903.bam"

In [36]:
!samtools view -c $p

19183425


In [80]:
!samtools idxstats $p | cut -f 1,3

1	851182
2	869755
3	728084
4	627123
5	652944
6	684057
7	559402
8	570709
9	473974
10	525235
11	492523
12	500876
13	352865
14	332676
15	319390
16	317600
17	273311
18	301593
19	178823
20	259496
21	139985
22	140093
X	266709
Y	158005
MT	27922
GL000207.1	4
GL000226.1	944
GL000229.1	39
GL000231.1	56
GL000210.1	34
GL000239.1	43
GL000235.1	32
GL000201.1	47
GL000247.1	38
GL000245.1	22
GL000197.1	35
GL000203.1	25
GL000246.1	41
GL000249.1	31
GL000196.1	22
GL000248.1	51
GL000244.1	35
GL000238.1	54
GL000202.1	43
GL000234.1	215
GL000232.1	120
GL000206.1	43
GL000240.1	53
GL000236.1	24
GL000241.1	62
GL000243.1	106
GL000242.1	91
GL000230.1	26
GL000237.1	86
GL000233.1	29
GL000204.1	101
GL000198.1	56
GL000208.1	145
GL000191.1	119
GL000227.1	92
GL000228.1	185
GL000214.1	417
GL000221.1	182
GL000209.1	116
GL000218.1	261
GL000220.1	2241
GL000213.1	100
GL000211.1	212
GL000199.1	713
GL000217.1	191
GL000216.1	228
GL000215.1	143
GL000205.1	238
GL000219.1	185
GL000224.1	305
GL000223.1	208
GL000195.1	268
GL000212.1

In [72]:
!samtools view $p | head -n 1000000 | cut -f 10 | perl -ne 'chomp;print length($_) . "\n"' | sort | uniq -c

samtools view: writing to standard output failed: Broken pipe
samtools view: error closing standard output: -1
   5352 10
   8518 11
  11644 12
  12930 13
  13457 14
  13995 15
  13001 16
  12617 17
  12500 18
  11928 19
  11098 20
   9612 21
  10351 22
   9831 23
   8254 24
   6235 25
   4218 26
   2682 27
   1772 28
   1265 29
   1092 30
   1108 31
   1134 32
   1116 33
   1147 34
   1261 35
   1400 36
   1454 37
   1732 38
   1784 39
   2071 40
   2270 41
   2483 42
   2536 43
   2506 44
   2552 45
   2816 46
   3052 47
   3283 48
   3347 49
      3 5
   3890 50
   3794 51
   4026 52
   4220 53
   4277 54
   4284 55
   4363 56
   4624 57
   4726 58
   5032 59
      9 6
   5208 60
   5117 61
   5454 62
   5491 63
   5705 64
   5475 65
   5618 66
   5617 67
   5779 68
   6032 69
    179 7
   6707 70
   6872 71
   9782 72
  21733 73
 630749 76
   1037 8
   2793 9
